In [1]:
import streamlit as st
import pandas as pd
from PIL import Image
import pylab as plt
import webbrowser
import base64
import io

st.title('Flick Pick')

2023-05-30 12:36:52.499 
  command:

    streamlit run c:\Users\javir\anaconda3\envs\ETL\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()